<a href="https://colab.research.google.com/github/shajarian/Music-Recommender-PySpark/blob/main/musicRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Music Recommender**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max
from pyspark.ml.feature import  StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

spark = SparkSession.builder.appName("lastfm").getOrCreate()

In [ ]:
!ls "/content/drive/My Drive/Semester 3/Big Data/HW4"

 archive.zip  'Assignment-4(1).pdf'   listenings.csv  'wordcount.ipynb - Colab.pdf'


In [ ]:
file_path = "/content/drive/My Drive/Semester 3/Big Data/HW4/listenings.csv"

In [ ]:
df_listenings = spark.read.format('csv').option('header',True).option('inferSchema',True).load(file_path) #data frame, header will infer column types from csv
df_listenings.show()

+-----------+-------------+--------------------+---------------+--------------------+
|    user_id|         date|               track|         artist|               album|
+-----------+-------------+--------------------+---------------+--------------------+
|000Silenced|1299680100000|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|1299679920000|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|1299679440000|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|1299679200000|            Acapella|          Kelis|            Acapella|
|000Silenced|1299675660000|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|1297511400000|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|1294498440000|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|1292438340000|               ObZen|      Meshuggah|               ObZen|
|000Silenced|1292437740000|   Yama's Messengers|      

Cleaning Table

In [ ]:
df_listenings = df_listenings.drop('date')#drop date and time col from data frame
df_listenings = df_listenings.na.drop() # removes null values in the row
df_listenings.show()

+-----------+--------------------+---------------+--------------------+
|    user_id|               track|         artist|               album|
+-----------+--------------------+---------------+--------------------+
|000Silenced|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|            Acapella|          Kelis|            Acapella|
|000Silenced|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|               ObZen|      Meshuggah|               ObZen|
|000Silenced|   Yama's Messengers|         Gojira|The Way of All Flesh|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For No...|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For

In [ ]:
rows = df_listenings.count()
cols = len(df_listenings.columns)
print(rows,cols)

13758905 4


In [ ]:
#in order to make a model, we need to know how many times a user has listened to each song
df_listenings_agg = df_listenings.select('user_id', 'track').groupby('user_id', 'track').agg(count('*').alias('count')).orderBy('user_id')
df_listenings_agg.show()

+-------+--------------------+-----+
|user_id|               track|count|
+-------+--------------------+-----+
| --Seph| White Winter Hymnal|    3|
| --Seph|         The Funeral|    1|
| --Seph|Hope There's Someone|    1|
| --Seph|         The Painter|    1|
| --Seph|          Je te veux|    1|
| --Seph|            War Pigs|    1|
| --Seph|                 F12|    1|
| --Seph|                Team|    1|
| --Seph|          Nightmares|    1|
| --Seph|               Radio|    1|
| --Seph|   All I Want Is You|    1|
| --Seph|    Little by Little|    2|
| --Seph|        After Nature|    1|
| --Seph|In the Hall of th...|    1|
| --Seph|   Hey There Delilah|    1|
| --Seph|   Let's Call It Off|    1|
| --Seph|               Leloo|    1|
| --Seph|             Pack Up|    1|
| --Seph|           Introitus|    1|
| --Seph|        The Leanover|    1|
+-------+--------------------+-----+
only showing top 20 rows



In [ ]:
row = df_listenings_agg.count()
col = len(df_listenings_agg.columns)
print(row,col)

9930128 3


In [ ]:
#we want to decrement our row size, its almost 10 million, lets reduce it inorder to process our data much faster
df_listenings_agg = df_listenings_agg.limit(20000)

In [ ]:
#We want to use StringIndexer to convert userID and track to unique, integer values
#StringIndexer encodes a string column of labels to a column of label indices
old_strindexer = [StringIndexer(inputCol = col, outputCol = col + '_index').fit(df_listenings_agg) for col in list(set(df_listenings_agg.columns)- set(['count']))]
indexer = [curr_strindexer.setHandleInvalid("keep") for curr_strindexer in old_strindexer]
pipeline = Pipeline(stages = indexer)
#the dataframe will be called data
#Transformers convert one dataframe into another either by updating the current values of a particular column (like converting categorical columns to numeric) or mapping it to some other values by using a defined logic.
#An Estimator implements the fit() method on a dataframe and produces a model.
data = pipeline.fit(df_listenings_agg).transform(df_listenings_agg)
data.show()

+-------+--------------------+-----+-----------+-------------+
|user_id|               track|count|track_index|user_id_index|
+-------+--------------------+-----+-----------+-------------+
| --Seph|          Nightmares|    1|    10600.0|         69.0|
| --Seph|Virus (Luke Fair ...|    1|    15893.0|         69.0|
| --Seph|Airplanes [feat H...|    1|      521.0|         69.0|
| --Seph|Belina (Original ...|    1|     3280.0|         69.0|
| --Seph|              Monday|    1|      334.0|         69.0|
| --Seph|Hungarian Dance No 5|    1|     7555.0|         69.0|
| --Seph|       Life On Mars?|    1|     1164.0|         69.0|
| --Seph|  California Waiting|    1|      195.0|         69.0|
| --Seph|       Phantom Pt II|    1|     1378.0|         69.0|
| --Seph|   Summa for Strings|    1|    13737.0|         69.0|
| --Seph|      Hour for magic|    2|     7492.0|         69.0|
| --Seph|Hungarian Rhapsod...|    1|     7556.0|         69.0|
| --Seph|     The Way We Were|    1|    14958.0|       

In [ ]:
data = data.select('user_id_index', 'track_index', 'count').orderBy('user_id_index')
data.show()

+-------------+-----------+-----+
|user_id_index|track_index|count|
+-------------+-----------+-----+
|          0.0|    10628.0|    1|
|          0.0|     3338.0|    1|
|          0.0|    12168.0|    1|
|          0.0|    11626.0|    2|
|          0.0|    10094.0|    4|
|          0.0|      427.0|    1|
|          0.0|    16878.0|    1|
|          0.0|    11722.0|    1|
|          0.0|    15074.0|    1|
|          0.0|     1359.0|    1|
|          0.0|     5874.0|    1|
|          0.0|    11184.0|    1|
|          0.0|     2372.0|    2|
|          0.0|    14316.0|    1|
|          0.0|     5346.0|    1|
|          0.0|    11194.0|    1|
|          0.0|     2241.0|    1|
|          0.0|     2864.0|    1|
|          0.0|     2663.0|    4|
|          0.0|     6064.0|    1|
+-------------+-----------+-----+
only showing top 20 rows



In [ ]:
(training, test) = data.randomSplit([0.5,0.5])
USERID = "user_id_index"
TRACK = "track_index"
COUNT = "count"

als = ALS(maxIter = 5, regParam = 0.01, userCol = USERID, itemCol = TRACK, ratingCol = COUNT)
# Alternating Least Squares algorithm
print(als)
model = als.fit(training)

predictions = model.transform(test)

ALS_899f6622d595


In [ ]:
recs = model.recommendForAllUsers(10)
recs.show()
recs.take(1)

+-------------+--------------------+
|user_id_index|     recommendations|
+-------------+--------------------+
|            0|[{8819, 7.976172}...|
|            1|[{9498, 13.427008...|
|            2|[{14825, 9.693659...|
|            3|[{15790, 10.14509...|
|            4|[{11940, 28.34658...|
|            5|[{308, 6.6717935}...|
|            6|[{9498, 7.9110384...|
|            7|[{15790, 12.23652...|
|            8|[{9498, 8.8175955...|
|            9|[{9498, 9.9392605...|
|           10|[{1739, 11.89157}...|
|           11|[{308, 8.949628},...|
|           12|[{11940, 18.02002...|
|           13|[{11940, 21.05193...|
|           14|[{11940, 32.46900...|
|           15|[{1739, 5.902044}...|
|           16|[{11940, 52.92298...|
|           17|[{1739, 9.112538}...|
|           18|[{15450, 10.09848...|
|           19|[{11940, 21.3478}...|
+-------------+--------------------+
only showing top 20 rows



[Row(user_id_index=0, recommendations=[Row(track_index=8819, rating=7.976171970367432), Row(track_index=15450, rating=5.884303569793701), Row(track_index=9498, rating=5.561005115509033), Row(track_index=3345, rating=5.289620399475098), Row(track_index=5041, rating=4.981709003448486), Row(track_index=1544, rating=4.981709003448486), Row(track_index=6153, rating=4.623381614685059), Row(track_index=12398, rating=4.533960342407227), Row(track_index=12890, rating=4.4961652755737305), Row(track_index=11688, rating=4.087423324584961)])]